#Proposta de Projeto

Com base nos dados fornecidos do Tribunal Regional Federal da 5ª Região, e tendo como ponto de início os dados de protocolo de um processo, treinar um algoritmo de regressão de modo a supor quanto tempo aquele processo irá durar. 

As <i>features</i> a serem utilizadas serão encontradas ao longo do desenvolvimento, assim como a melhor métricas para validar esse sistema e quais foram os processos utilizados. 

Ademais, o trabalho a seguir usará a seguinte estrutura:


*  Inicialização e leitura dos dados
*  Escolha e separação das features
*  Escolha das métricas
*  Teste primário
*  Melhoria do modelo

OBS: Essa proposta de projeto foi posteriormente suspensa, de modo que esse notebook corresponde apenas com as tentativas iniciais de desenvolvimento.




##Inicialização e leitura dos dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/My Drive/Dados IC/IC.zip'

Archive:  /content/drive/My Drive/Dados IC/IC.zip
replace PJe-JF.sqlite3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [ ]:
#Conexão e leitura do arquivo PJe-JF.sqlite3
#Seções Judiciárias

connection = sqlite3.connect('/content/PJe-JF.sqlite3')
df_al = pd.read_sql_query("SELECT * from AL", connection)
df_ce = pd.read_sql_query("SELECT * from CE", connection)
df_pb = pd.read_sql_query("SELECT * from PB", connection)
df_pe = pd.read_sql_query("SELECT * from PE", connection)
df_rn = pd.read_sql_query("SELECT * from RN", connection)
df_se = pd.read_sql_query("SELECT * from SE", connection)

data_jf = [df_al, df_ce, df_pb, df_pe, df_rn, df_se]
df_jf = pd.concat(data_jf)

In [ ]:
#Conexão e leitura do arquivo PJe-TRF.sqlite3
#Segundo Grau
connection = sqlite3.connect('/content/PJe-TRF.sqlite3')

df_proc = pd.read_sql_query("SELECT * from processos", connection)
df_dec = pd.read_sql_query("SELECT * from decisoes", connection)
df_mov = pd.read_sql_query("SELECT * from movimentacoes", connection)


In [ ]:
#Limpeza dos dados nulos 
for d in data_jf:
  d.drop(d.index[d['num_processo'] == '0'], inplace = True)

df_jf.drop(df_jf.index[df_jf['num_processo'] == '0'], inplace = True)

##Recolhimento e separação dos dados

Todas as informações necessárias encontravam-se espalhadas, cada um em uma tabela diferente. 
Os dados sobre as datas de encerramento dos processos estavam em df_dec,as datas de distribuições dos processos foram obtidas na tabela de movimentação (df_mov) e todas as outras colunas que apresentam características desses processos (polos, classe judicial, assunto, etc) estavam nas tabelas df_jf e df_proc. 

In [ ]:
#Separa, no df das decisões, apenas os processos encerrados (já sentenciados)
df_sentencas = df_dec[df_dec['documento'].str.contains("Sentença")]

#Separa no df das movimentações os números dos processos que estão em 1º Grau
proc_grau1 = (df_mov[df_mov["aplicacao"].str.contains("1º")]["num_processo"]).unique()

#Filtra para obter apenas os processos sentenciados e de 1º Grau
df_sentencas = df_sentencas[df_sentencas["num_processo"].isin(proc_grau1)]

In [ ]:
#Separa em uma coluna a data de sentença do processo
dfc_sentencas = df_sentencas.copy()
datas_sentencas = []

for movimentacao in dfc_sentencas['documento']:
  data_d = movimentacao.split(" ")[0]
  datas_sentencas.append(data_d)

dfc_sentencas['data_sentenca'] = datas_sentencas
dfc_sentencas['data_sentenca'] = pd.to_datetime(dfc_sentencas['data_sentenca'])

#Ordena os dados por número de processo (do menor pro maior) e depois por data
# de sentença (do maior pro menor) e pega a primeira linha pra cada número de processo
# Ou seja, pega a mais recente das decisões pra cada processo.
df_sentencas_sort = dfc_sentencas.sort_values(['num_processo', 'data_sentenca'], ascending=[True, False])
df_sentencas = df_sentencas_sort.groupby('num_processo').first().reset_index()


In [ ]:
#Separa, no df das movimentações, os processos de 1º Grau e já sentencias
df_mov = df_mov[df_mov["aplicacao"].str.contains("1º")]
df_mov_penc = df_mov[df_mov['num_processo'].isin(list(df_sentencas['num_processo']))]

datas_mov = []
#Separa e transforma em datetime as datas de movimentação dos processos
for movimentacao in df_mov_penc['movimento']:
  data_d = movimentacao.split(" ")[0]
  datas_mov.append(data_d)

df_mov_processos_encerrados = df_mov_penc.copy()
df_mov_processos_encerrados['data_mov'] = datas_mov
df_mov_processos_encerrados['data_mov'] = pd.to_datetime(df_mov_processos_encerrados['data_mov'])

#Ordena os dados por número de processo e depois por data de movimentação 
# (do menor pro maior) pega a primeira linha pra cada número de processo
# Ou seja, pega o movimento mais antigo para cada processo.
df_mov_sort = df_mov_processos_encerrados.sort_values(['num_processo', 'data_mov'], ascending=[True, True])
df_distrib_processos_encerrados = df_mov_sort.groupby('num_processo').first().reset_index()


In [ ]:
#Une os dois datasets (distribuição e sentença) em um só
df_distrib_sentenca = pd.merge(df_sentencas, df_distrib_processos_encerrados, on='num_processo')

In [ ]:
#Unindo as tabelas que contém os dados textuais sobre os processos
# através do numero de processo e removendo as entradas duplicadas
data_merged = pd.merge(left=df_jf,right=df_proc,  left_on='num_processo', right_on='num_processo')
new_data = data_merged[data_merged.duplicated(keep='first') == False]

In [ ]:
#Separa, no dataset dos dados sobre os processos, apenas os processos que já aparecem
# como encerrados no df de decisões.
df_processos_selecionados = new_data[new_data['num_processo'].isin(list(df_sentencas['num_processo']))]

In [ ]:
#Une os dados previamento selecionados (os já encerrados) com os
# seus respectivos dados, que estavam guardados em outra tabela.  
df_processos_encerrados = pd.merge(df_processos_selecionados, df_distrib_sentenca, on='num_processo')
df_processos_encerrados.rename(columns={"data_mov": "data_distribuicao"}, inplace=True)
df_processos_encerrados.columns

Index(['num_processo', 'classe', 'titular_substituto', 'vara', 'uf',
       'polo_ativo_x', 'polo_passivo_x', 'assunto_x', 'ultimo_evento',
       'polo_ativo_y', 'polo_passivo_y', 'data_distrib', 'orgao_julgador',
       'classe_judicial', 'assunto_y', 'assuntos_tabela', 'url_atual',
       'fonte_pagina', 'documento_x', 'html', 'data_sentenca', 'movimento',
       'documento_y', 'aplicacao', 'data_distribuicao'],
      dtype='object')

In [ ]:
df_processos = df_processos_encerrados[['num_processo', 'classe', 'titular_substituto', 'vara', 'uf',
                                        'polo_ativo_x', 'polo_passivo_x', 'assunto_x', 'ultimo_evento',
                                        'polo_ativo_y', 'polo_passivo_y', 'data_distrib', 'orgao_julgador',
                                        'classe_judicial', 'assunto_y', 'assuntos_tabela', 
                                        'data_sentenca', 'data_distribuicao']].copy()
df_processos.head(3)

In [ ]:
direitos = []
assunto_especifico = []

for assunto in df_processos['assuntos_tabela']:
  assuntos = assunto.split("|")
  direitos.append(assuntos[0])
  assunto_especifico.append(assuntos[-1])


In [ ]:
df_processos['tempo_processual'] = df_processos['data_sentenca'] - df_processos["data_distribuicao"]
df_processos['tipo_direito'] = direitos
df_processos['assunto_especifico'] = assunto_especifico

df_processos.head()

,num_processo,classe,titular_substituto,vara,uf,polo_ativo_x,polo_passivo_x,assunto_x,ultimo_evento,polo_ativo_y,polo_passivo_y,data_distrib,orgao_julgador,classe_judicial,assunto_y,assuntos_tabela,data_sentenca,data_distribuicao,tempo_processual,tipo_direito,assunto_especifico
0,0811659-57.2017.4.05.8000,PROCEDIMENTO COMUM,S,4ª VARA FEDERAL,AL,AUTOR LOURDES DE ARAUJO SILVA SOUZA,RÉU INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,Remetidos os Autos (em grau de recurso) para TRF5,LOURDES DE ARAUJO SILVA SOUZA\nAPELANTE\n L...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,10/01/2019,Gab 5 - Des. ROBERTO MACHADO,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,2018-05-07,2017-12-29,129 days,DIREITO PREVIDENCIÁRIO,Concessão
1,0811659-57.2017.4.05.8000,PROCEDIMENTO COMUM,S,4ª VARA FEDERAL,AL,AUTOR LOURDES DE ARAUJO SILVA SOUZA,RÉU INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,Remetidos os Autos (em grau de recurso) para TRF5,LOURDES DE ARAUJO SILVA SOUZA\nAPELANTE\n L...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,10/01/2019,Gab 5 - Des. ROBERTO MACHADO,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,2018-05-07,2017-12-29,129 days,DIREITO PREVIDENCIÁRIO,Concessão
2,0811658-72.2017.4.05.8000,PROCEDIMENTO COMUM,S,1ª VARA FEDERAL,AL,AUTOR MARIA BENEDITA FERREIRA DOS SANTOS,RÉU INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,Remetidos os Autos (em grau de recurso) para TRF5,MARIA BENEDITA FERREIRA DOS SANTOS\nAPELANTE\n...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,10/05/2018,Gab 8 - Des. FERNANDO BRAGA,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,2018-04-18,2017-12-29,110 days,DIREITO PREVIDENCIÁRIO,RMI sem incidência de Teto Limitador
3,0811658-72.2017.4.05.8000,PROCEDIMENTO COMUM,S,1ª VARA FEDERAL,AL,AUTOR MARIA BENEDITA FERREIRA DOS SANTOS,RÉU INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,Remetidos os Autos (em grau de recurso) para TRF5,MARIA BENEDITA FERREIRA DOS SANTOS\nAPELANTE\n...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,10/05/2018,Gab 8 - Des. FERNANDO BRAGA,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,2018-04-18,2017-12-29,110 days,DIREITO PREVIDENCIÁRIO,RMI sem incidência de Teto Limitador
4,0811657-87.2017.4.05.8000,PROCEDIMENTO COMUM,S,2ª VARA FEDERAL,AL,AUTOR GILBERTO FIRMINO DOS SANTOS,RÉU INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,Juntada de Certidão de Intimação,GILBERTO FIRMINO DOS SANTOS\nAPELANTE\n LUI...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,20/08/2018,Gab 13 - Des. ROGÉRIO FIALHO MOREIRA,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,DIREITO PREVIDENCIÁRIO|Pedidos Genéricos Relat...,2018-01-06,2017-12-29,8 days,DIREITO PREVIDENCIÁRIO,Concessão


In [ ]:
df_processos['tempo_processual'] = df_processos['tempo_processual'].astype(str)
df_processos['tempo_processual'] = df_processos['tempo_processual'].str.extract('(\d+)', expand=False)
df_processos['tempo_processual'] = df_processos['tempo_processual'].astype(float)


###OUTROS

In [ ]:
df_processos_copy = df_processos.copy()

# Existe um total de 46214 linhas do df, de forma que o número 462 foi
# escolhido de modo a agrupar todos os assuntos que apareciam com
# frequencia menor que 1% (ou de 0.5%, como no primeiro caso)
df_processos_copy.loc[df_processos_copy['assunto_especifico'].value_counts()
  [df_processos_copy['assunto_especifico']].values < 462, 'assunto_especifico'] = "Outros"

df_processos_copy.loc[df_processos_copy['classe'].value_counts()
  [df_processos_copy['classe']].values < 462, 'classe'] = "OUTRA"  

In [ ]:
df_processos_copy.columns

Index(['num_processo', 'classe', 'titular_substituto', 'vara', 'uf',
       'polo_ativo_x', 'polo_passivo_x', 'assunto_x', 'ultimo_evento',
       'polo_ativo_y', 'polo_passivo_y', 'data_distrib', 'orgao_julgador',
       'classe_judicial', 'assunto_y', 'assuntos_tabela', 'data_sentenca',
       'data_distribuicao', 'tempo_processual', 'tipo_direito',
       'assunto_especifico'],
      dtype='object')

##Train and Test

In [ ]:
df_processos = df_processos_copy.copy()
df = df_processos[['assunto_y', 'uf', 'tempo_processual']]

In [ ]:
x_values = df_processos[['assunto_y', 'uf']]
x_values = pd.get_dummies(x_values, ['assunto_y', 'uf'], drop_first=True).values

y_values = (df_processos['tempo_processual']).values
y_values = y_values.reshape(-1,1)

df_values = np.concatenate((x_values, y_values), axis=1)

print(x_values.shape)
print(y_values.shape)
print(df_values.shape)

(44734, 749)
(44734, 1)
(44734, 750)


In [ ]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
df_values_norm = normalizer.fit_transform(df_values)
x_values_norm = normalizer.fit_transform(x_values)
y_values_norm = normalizer.fit_transform(y_values)


In [ ]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x_values, y_values, test_size = 0.3, random_state = 1)
print(xTrain.shape)
print(yTrain.shape)

(31313, 749)
(31313, 1)


##Linear Regression

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(xTrain, yTrain)
time_predict = reg.predict(xTest)

yTest = yTest.reshape(-1, 1)
#time_predict = time_predict.reshape(-1, 1)

print("Mean Squared Error:", mean_squared_error(yTest, time_predict))
print("Coeficiente de determinação", r2_score(yTest, time_predict))


Mean Squared Error: 1.6457203951060405e+25
Coeficiente de determinação -1.7711865721780755e+20


In [ ]:
#Usando Cross-Validation

reg = linear_model.LinearRegression()
cv_score = cross_val_score(reg, x_values, y_values, cv=4)  # k = 4

cv_score

array([-9.12472186e+23, -1.16535603e+19, -3.28476152e+20, -9.20611968e+10])

###Sobre as métricas

Existe um momento na separação dos dados que optei por agrupar classes e assuntos específicos que aparecessem no dataframe com uma frequência menor do que 1%. Essas classes e assuntos foram agrupados com os títulos de "OUTRA" e "Outros", respectivamente. 

Quando o modelo é treinado com essa parte do código funcionando, temos os seguintes resultados: <br/>
Mean Squared Error: 152682.49874786718 <br/>
R2: 0.3251096962283918

E quanto o modelo é treinado com essa opção desabilitada, temos o seguinte: <br/>
Mean Squared Error: 3.697330763076892e+19 </br>
Coeficiente de determinação -163430170602438.94

##Ridge Regression


In [ ]:
alphas = [0.1, 0.2, 0.3]
reg = linear_model.RidgeCV(alphas=alphas)

reg.fit(xTrain, yTrain)
#print('Coefficients: {}\n'.format(repr(reg.coef_)))
print('Intercept: {}\n'.format(reg.intercept_))
print('Chosen alpha: {}\n'.format(reg.alpha_))


r2 = r2_score(yTest, time_predict)
print("Mean Squared Error:", mean_squared_error(yTest, time_predict))
print('R2: {}\n'.format(r2))

Intercept: [460.77508452]

Chosen alpha: 0.2

Mean Squared Error: 83875.38425048719
R2: 0.09730137171225217

